# Модуль ```pickle```

Модуль ```pickle``` реализует двоичные протоколы для сериализации и десериализации Python объектов. «Маринование» - это процесс, посредством которого Python объекты преобразуется в поток байтов, а «размаринование» - это обратная операция, посредством которой поток байтов (из двоичного файла или байтового объекта) преобразуется обратно в объекты.

Протокол, используемый в ```pickle```, не просто записывает байты в файл или строку, он записывает дополнительную метаинформацию о типе объекта. Без этого будет невозможна десериализация этого байтового представления.

In [1]:
import pickle

Интерфейс, предоставляемый модулем ```pickle```, схож с интерфейсом модуля ```json```. Вот краткое описание:
- ```dump(obj, file)``` - функция сериализации объекта ```obj``` в бинарный файл ```file```; 
- ```dumps(obj)``` - функция сериализации объекта ```obj``` в бинарную строку;
- ```load(file)``` - функция десериализации объекта из бинарного файла ```file```;
- ```loads(data)``` - функция десериализации объекта из бинарной строки ```data```.

In [2]:
# подготовим данные для сериализации
d_1 = 'бнопня'
d_2 = 196.
d_3 = {
    'foo': [1, 2, 3],
    'bar': ('a', 'b', 'c'),
    'baz': {True, False, None},
}

In [3]:
# сериализация
serialized_str = pickle.dumps(d_1)
serialized_float = pickle.dumps(d_2)
serialized_dict = pickle.dumps(d_3)

print(f'{type(serialized_str) = }')

print(f'{type(d_1)} -> {serialized_str}')
print(f'{type(d_2)} -> {serialized_float}')
print(f'{type(d_3)} -> {serialized_dict}')

type(serialized_str) = <class 'bytes'>
<class 'str'> -> b'\x80\x04\x95\x10\x00\x00\x00\x00\x00\x00\x00\x8c\x0c\xd0\xb1\xd0\xbd\xd0\xbe\xd0\xbf\xd0\xbd\xd1\x8f\x94.'
<class 'float'> -> b'\x80\x04\x95\n\x00\x00\x00\x00\x00\x00\x00G@h\x80\x00\x00\x00\x00\x00.'
<class 'dict'> -> b'\x80\x04\x956\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x03foo\x94]\x94(K\x01K\x02K\x03e\x8c\x03bar\x94\x8c\x01a\x94\x8c\x01b\x94\x8c\x01c\x94\x87\x94\x8c\x03baz\x94\x8f\x94(\x89\x88N\x90u.'


In [4]:
# десериализация

print(f'Десериализация строки: {pickle.loads(serialized_str)}')
print(f'Десериализация числа: {pickle.loads(serialized_float)}')
print(f'Десериализация словаря: {pickle.loads(serialized_dict)}')

Десериализация строки: бнопня
Десериализация числа: 196.0
Десериализация словаря: {'foo': [1, 2, 3], 'bar': ('a', 'b', 'c'), 'baz': {False, True, None}}


In [5]:
# не самый красивый способ создать функцию с двумя параметрами
def constructor_with_two_args(name, k):
    # болванка
    def bar(a, b):
        pass
    # копирование всех нужных атрибутов
    bar.__name__ = name
    for attr_name, v in k.items():
        setattr(bar, attr_name, v)
    # возврат новой/старой функции
    return bar

class MyPickler(pickle.Pickler):
    def reducer_override(self, obj):
        if getattr(obj, "__name__", None) == "foo":
            return constructor_with_two_args, (obj.__name__, 
                                               {'__doc__': obj.__doc__, '__code__': obj.__code__})
        else:
            return NotImplemented

In [5]:
import io

def foo(a, b):
    """Функция-подопытный"""
    print(f'{a + b = }')

# будем писать в байтовый поток
f = io.BytesIO()
# создаем собственный сериализатор/десериализатор
pickler = MyPickler(f)

# дампим функцию в бинарный поток
pickler.dump(foo)

# для чистоты эксперимента удалим функцию
del foo

# размаринуем функцию
baz = pickle.loads(f.getvalue())

print(f'{baz = }')

baz(1, 2)  # Алоха!

baz = <function constructor_with_two_args.<locals>.bar at 0x0000016EE1F035E0>
a + b = 3


# Сравнение ```pickle``` и других протоколов сериализации и десериализации

Для понимания отличий маринования от прочих протоколов рассмотрим небольшое сравнение (выдержка из документации).

## ```pickle``` vs ```marshal```

В Python реализован еще один модуль для двоичной сериализации и десериализации ```marshal```. Это более низкоуровневый протоко по сравнению с ```pickle```. Его использование не рекомендуется, а предназначение заключается в поддержке файлов ```.pyc```.

- Модуль ```pickle``` позволяет отслеживать уже сериализованные объекты. Это позволяет легко работать с объектами содержащими рекурсивные ссылки. Модуль ```marshal``` этого не умеет и при попытке сериализовать объект с циклическими ссылками есть высокая вероятность сломать интерпретатор.
- Модуль ```pickle``` умеет работать с пользовательскими типами данных (классами), при условии, что его объявление находится в той же области видимости. ```marshal``` не имеет такой функциональности. 
- В связи с тем, что ```marshal``` предназначен сугубо для внутреннего использования, переносимость между разными версиями интерпретатора не гарантируется.

## ```pickle``` vs ```json```

- ```json``` - это **текстовый** формат, а ```pickle``` - это двоичный формат;
- ```json``` удобочитаем, а ```pickle``` - нет; 
- ```json``` совместим и широко используется вне экосистемы Python, в то время как ```pickle``` специфичен для Python; 
- ```json``` по умолчанию может представлять только ограниченное количество **встроенных** типов Python и не может сериализовать пользовательские классы;
- ```pickle``` может обрабатывать чрезвычайно большое количество типов Python (многие из них автоматически, благодаря умному использованию средств самоанализа Python; поддержка сложных типов может быть добавлена путем реализации конкретного API);
- в отличие от ```pickle```, десериализация ненадежного ```json``` сама по себе не создает уязвимости выполнения произвольного кода.

Дополнительно стандартная библиотека Python содержит модуль ```shelve```, который использует ```pickle``` для реализации простейшей DBM-подобной базы данных. Структура такой базы данных представлена в Python как объект типа ```dict```.

# Полезные ссылки

- [Документация по модулю ```pickle```](https://docs.python.org/3/library/pickle.html)
- [Документация по модулю ```shelve```](https://docs.python.org/3/library/shelve.html#module-shelve)
- [Пример динамического создания функции в Python](https://gist.github.com/dhagrow/d3414e3c6ae25dfa606238355aea2ca5)